# Experiment with different distance metrics & clustering algorithms

In [1]:
import sys
import logging

nblog = open("nb.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [2]:
import time
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics.cluster import normalized_mutual_info_score

import data_loader
import pp
import clustering

import scanpy as sc


%load_ext autoreload
%autoreload 2

In [3]:
# Local path 
DATA_PATH = '/projects/zhanglab/users/johnson/data/GSE72056_processed.h5ad'

# Loading raw data
adata = data_loader.load_annData(DATA_PATH)    
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 4645 × 22287
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'log1p'

## Data Preprocessing

In [4]:
# Preprocess data # pp.preprocess(adata)
pp.filter_cell(adata, min_genes = 1500)
pp.filter_gene(adata, min_cells = 10)
pp.filter_mt_genes(adata)
pp.normalize_size(adata, target_sum = 1e4)
pp.normalize_log(adata)
pp.highly_variable_genes(adata, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
adata

/home/yenhsian/miniconda3/lib/python3.11/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


AnnData object with n_obs × n_vars = 4644 × 21120
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

## Clustering

### Leidon

In [5]:
# PCA
clustering.pca(adata, svd_solver='arpack')
# KNN
clustering.neighbors(adata, n_neighbors = 10, use_pca = True)
# Leiden-clustering based on PCA
clustering.cluster_leiden(adata)
# UMAP embedding based on KNN
sc.tl.umap(adata)

leiden completed, time=1.8s


### K-medoids

#### true labels

parse true labels from cell's name

In [6]:
# Classification based on the name of data
from collections import Counter

def encode(celltypes):
    labels_dict = {}
    labels = []
    cur = 0

    for c in celltypes:
        if c not in labels_dict:
            labels_dict[c] = cur
            cur += 1 # increase smallest index
        labels.append(labels_dict[c])
    return labels, labels_dict


celltypes = pd.Series(adata.obs_names).apply(lambda x : x[:4].upper())
labels, labels_dict = encode(celltypes)

adata.obs['true_label'] = labels
labels_dict

{'CY72': 0,
 'CY58': 1,
 'CY71': 2,
 'CY81': 3,
 'CY80': 4,
 'CY74': 5,
 'CY79': 6,
 'CY82': 7,
 'CY53': 8,
 'CY59': 9,
 'CY67': 10,
 'CY65': 11,
 'CY78': 12,
 'CY84': 13,
 'CY60': 14,
 'CY88': 15,
 'CY89': 16,
 'CY75': 17,
 'CY94': 18,
 'SS2_': 19,
 'MONI': 20}

#### Clustering with different metrics

In [ ]:
from banditpam import KMedoids
from sklearn.metrics.cluster import normalized_mutual_info_score 

df_score = pd.DataFrame()
df_score.index.name = 'k'

for metric in ['L1', 'L2', 'inf', 'cos', 'manhattan']:

    score_list = []
    for k in range(1, 9):
        clustering.cluster_banditpam(adata, n_medoids = k, metric = metric)
        nmi = normalized_mutual_info_score(adata.obs['true_label'], adata.obs['kmed_'+metric])
        score_list.append(nmi)
    df_score[metric] = score_list

df_score.index = range(1,9)
df_score

kmed_L1 (K = 1) completed, time=30.7s
kmed_L1 (K = 2) completed, time=109.4s
kmed_L1 (K = 3) completed, time=66.9s
kmed_L1 (K = 4) completed, time=152.7s
kmed_L1 (K = 5) completed, time=174.9s


#### Find best K

In [ ]:
import matplotlib.pyplot as plt
colormap = plt.get_cmap('tab10')

for i, metric in enumerate(['L1', 'L2', 'inf', 'cos']):
    plt.plot(df_score.index, df_score[metric], label=metric, color=colormap(i))

plt.title('NMI v.s. number of K')
plt.xlabel('K')
plt.ylabel('NMI')
plt.legend()
plt.show()

In [ ]:
clustering.cluster_banditpam(adata, n_medoids = 5, metric = 'L1')
clustering.cluster_banditpam(adata, n_medoids = 7, metric = 'L2')
clustering.cluster_banditpam(adata, n_medoids = 7, metric = 'inf')
clustering.cluster_banditpam(adata, n_medoids = 6, metric = 'cos')
clustering.cluster_banditpam(adata, n_medoids = 6, metric = 'manhattan')

In [ ]:
adata.obs

#### Visualize clustering result 

In [ ]:
# True labels are stored in adata.obs
import warnings
warnings.filterwarnings('ignore')
sc.pl.umap(adata, color=['leiden', 'kmed_L1' , 'kmed_L2' , 'kmed_inf', 'kmed_cos', 'kmed_manhattan' ,'true_label'])

#### Benchmarking clustering

use NMI

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

print('leiden: ', normalized_mutual_info_score(adata.obs['true_label'], adata.obs['leiden']))
for metric in ['L1', 'L2', 'inf', 'cos', 'manhattan']:
    print(f"kmed {metric}: {normalized_mutual_info_score(adata.obs['true_label'], adata.obs['kmed_'+metric])}")